In [2]:
import os
import gc
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
DATA_DIR = './input/'

hex2dec = lambda x: int(x, 16)

In [3]:
train = pd.read_csv(DATA_DIR+'train.csv')

In [4]:
cols = ['ced6a7e91','9df4daa99','83c3779bf','edc84139a','f1e0ada11',
                '73687e512','aa164b93b','342e7eb03','cd24eae8a','8f3740670',
                '2b2a10857','a00adf70e','3a48a2cd2','a396ceeb9','9280f3d04',
                'fec5eaf1a','5b943716b','22ed6dba3','5547d6e11','e222309b0',
                '5d3b81ef8','1184df5c2','2288333b4','f39074b55','a8b721722',
                '13ee58af1','fb387ea33','4da206d28','ea4046b8d','ef30f6be5',
                'b85fa8b27','2155f5e16']
ordered_ids = ['a70090dc7','5b74875d4','ca6c4ea0d','242407fc6','7ff66b22b',
               'f43a48254','c9573f4ac','c14136677','08f544828','e77e32574',
               '61636fa41','4c0316d32','414871b56','2c518cd87','9eb93baef',
               'd63b2307d','db9ddeb12','7e45f5bd5','680ecfac8','814f58e7b',
               'eb850ef06','6edaf114d']

rows = []
for oi in ordered_ids:
    rows.append(train[train.ID == oi].index[0])


rows = np.array(rows)

In [5]:
tmp = train.loc[rows, ["ID","target"]+cols]
print('original shape', tmp.shape)
#tmp

original shape (22, 34)


In [6]:
df_cand_col = train.loc[rows, :]
df_cand_col = df_cand_col.iloc[:, 2:]
df_cand_col

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
2848,0.0,0,0.0,0,0,0,0,0,4900000.00,0.0,...,0.0,500000.0,0.00,0,0,480000,0,0,24048000,328000
1582,0.0,0,0.0,0,0,0,0,0,16733333.34,0.0,...,0.0,226500.0,200000.00,0,0,7900000,0,0,0,15172000
3509,0.0,0,0.0,0,0,0,0,0,922000.00,0.0,...,0.0,0.0,0.00,112000,0,0,0,0,0,8200000
626,0.0,0,0.0,0,0,0,0,0,2300000.00,0.0,...,0.0,550000.0,1800000.00,0,0,0,0,0,0,3260000
2191,0.0,0,0.0,0,0,0,0,0,356000.00,0.0,...,0.0,538000.0,2066666.66,0,0,0,0,0,0,0
4261,0.0,0,0.0,0,0,0,0,0,0.00,0.0,...,342000.0,0.0,0.00,16332000,0,0,0,0,0,0
3493,0.0,0,0.0,0,0,0,0,0,1838333.34,0.0,...,0.0,294000.0,0.00,6502000,0,0,0,0,0,0
3340,0.0,0,0.0,0,0,1940000,0,0,4870000.00,0.0,...,200000.0,0.0,4000000.00,0,0,9200000,0,0,0,4420000
156,0.0,0,0.0,0,0,740444,0,0,600000.00,0.0,...,0.0,0.0,0.00,0,0,0,0,0,0,0
4040,0.0,0,0.0,0,0,0,0,0,0.00,0.0,...,0.0,0.0,0.00,0,0,0,0,0,0,0


In [7]:
df_new = train.loc[rows, cols]

In [8]:
def bf_search(df_new, df_cand):
    cnt = 0
    head_curr = df_new.values[1:, 0]
    tail_curr = df_new.values[:-1, -1]
    while True:
        for c in df_cand.columns:
            if c in df_new:
                continue
            elif np.all(
                df_cand[c].iloc[:-1].values==head_curr
            ) and len(df_cand[c].unique())>1:
                df_new.insert(0, c, df_cand[c].values)
                head_curr = df_new.values[1:, 0]
                print(c, 'found head!', 'new shape', df_new.shape)
                cnt += 1
                break
            elif np.all(
                df_cand[c].iloc[1:].values==tail_curr
            ) and len(df_cand[c].unique())>1:
                df_new[c] = df_cand[c].values
                tail_curr = df_new.values[:-1, -1]
                print(c, 'found tail!', 'new shape', df_new.shape)
                cnt += 1
                break
            else:
                continue
        if cnt==0:
            break
        else:
            cnt = 0
            continue
    return df_new

In [9]:
print('Column searching ...')
df_new = bf_search(df_new, df_cand_col)

Column searching ...
794e93ca6 found tail! new shape (22, 33)
070f95c99 found tail! new shape (22, 34)
939f628a7 found tail! new shape (22, 35)
7e814a30d found tail! new shape (22, 36)
a6e871369 found tail! new shape (22, 37)
0dc4d6c7d found tail! new shape (22, 38)
bc70cbc26 found tail! new shape (22, 39)
aca228668 found tail! new shape (22, 40)


In [10]:
df_new

,ced6a7e91,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,cd24eae8a,8f3740670,...,b85fa8b27,2155f5e16,794e93ca6,070f95c99,939f628a7,7e814a30d,a6e871369,0dc4d6c7d,bc70cbc26,aca228668
2848,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,384000.00,2515000.00,...,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00,791200.00,334000.00
1582,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,384000.00,...,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00,791200.00
3509,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,...,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00
626,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,...,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34
2191,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,...,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00
4261,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,...,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00
3493,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,...,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66
3340,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,...,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00
156,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,...,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00
4040,371200.00,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,...,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00


In [11]:
df_new = df_new.T.copy()
df_new

,2848,1582,3509,626,2191,4261,3493,3340,156,4040,...,1125,774,2700,3742,3843,2162,1778,2217,4109,1918
ced6a7e91,900000.00,500000.00,0.00,831000.00,819500.00,1614909.10,1234666.66,2400000.00,236857.14,371200.00,...,137000.00,224000.00,660000.00,366666.66,108000.00,2530000.00,935000.00,1786666.66,1002666.66,50000.00
9df4daa99,380000.00,900000.00,500000.00,0.00,831000.00,819500.00,1614909.10,1234666.66,2400000.00,236857.14,...,164000.00,137000.00,224000.00,660000.00,366666.66,108000.00,2530000.00,935000.00,1786666.66,1002666.66
83c3779bf,1359000.00,380000.00,900000.00,500000.00,0.00,831000.00,819500.00,1614909.10,1234666.66,2400000.00,...,160000.00,164000.00,137000.00,224000.00,660000.00,366666.66,108000.00,2530000.00,935000.00,1786666.66
edc84139a,320000.00,1359000.00,380000.00,900000.00,500000.00,0.00,831000.00,819500.00,1614909.10,1234666.66,...,371200.00,160000.00,164000.00,137000.00,224000.00,660000.00,366666.66,108000.00,2530000.00,935000.00
f1e0ada11,1432000.00,320000.00,1359000.00,380000.00,900000.00,500000.00,0.00,831000.00,819500.00,1614909.10,...,236857.14,371200.00,160000.00,164000.00,137000.00,224000.00,660000.00,366666.66,108000.00,2530000.00
73687e512,0.00,1432000.00,320000.00,1359000.00,380000.00,900000.00,500000.00,0.00,831000.00,819500.00,...,2400000.00,236857.14,371200.00,160000.00,164000.00,137000.00,224000.00,660000.00,366666.66,108000.00
aa164b93b,565142.86,0.00,1432000.00,320000.00,1359000.00,380000.00,900000.00,500000.00,0.00,831000.00,...,1234666.66,2400000.00,236857.14,371200.00,160000.00,164000.00,137000.00,224000.00,660000.00,366666.66
342e7eb03,532000.00,565142.86,0.00,1432000.00,320000.00,1359000.00,380000.00,900000.00,500000.00,0.00,...,1614909.10,1234666.66,2400000.00,236857.14,371200.00,160000.00,164000.00,137000.00,224000.00,660000.00
cd24eae8a,384000.00,532000.00,565142.86,0.00,1432000.00,320000.00,1359000.00,380000.00,900000.00,500000.00,...,819500.00,1614909.10,1234666.66,2400000.00,236857.14,371200.00,160000.00,164000.00,137000.00,224000.00
8f3740670,2515000.00,384000.00,532000.00,565142.86,0.00,1432000.00,320000.00,1359000.00,380000.00,900000.00,...,831000.00,819500.00,1614909.10,1234666.66,2400000.00,236857.14,371200.00,160000.00,164000.00,137000.00


In [12]:
df_cand_row = train[df_new.index].T.copy()
df_cand_row.head()

,0,1,2,3,4,5,6,7,8,9,...,4449,4450,4451,4452,4453,4454,4455,4456,4457,4458
ced6a7e91,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,600000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,200000.0,0.0
9df4daa99,0.0,4000000.0,0.0,0.0,0.0,34400000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
83c3779bf,1600000.0,0.0,0.0,0.0,0.0,39000000.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18000000.0
edc84139a,0.0,0.0,0.0,0.0,0.0,13000000.0,2000000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10000000.0
f1e0ada11,0.0,0.0,0.0,0.0,0.0,0.0,2000000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
print('Row searching ...')
df_new = bf_search(df_new, df_cand_row)

Row searching ...


In [14]:
df_new = df_new.T.copy()
df_new

,ced6a7e91,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,cd24eae8a,8f3740670,...,b85fa8b27,2155f5e16,794e93ca6,070f95c99,939f628a7,7e814a30d,a6e871369,0dc4d6c7d,bc70cbc26,aca228668
2848,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,384000.00,2515000.00,...,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00,791200.00,334000.00
1582,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,384000.00,...,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00,791200.00
3509,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,...,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00
626,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,...,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34
2191,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,...,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00
4261,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,...,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00
3493,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,...,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66
3340,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,...,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00
156,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,...,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00
4040,371200.00,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,...,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00


In [15]:
df_cand_col = train.loc[df_new.index, :]
df_cand_col = df_cand_col.iloc[:, 2:]
df_cand_col

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
2848,0.0,0,0.0,0,0,0,0,0,4900000.00,0.0,...,0.0,500000.0,0.00,0,0,480000,0,0,24048000,328000
1582,0.0,0,0.0,0,0,0,0,0,16733333.34,0.0,...,0.0,226500.0,200000.00,0,0,7900000,0,0,0,15172000
3509,0.0,0,0.0,0,0,0,0,0,922000.00,0.0,...,0.0,0.0,0.00,112000,0,0,0,0,0,8200000
626,0.0,0,0.0,0,0,0,0,0,2300000.00,0.0,...,0.0,550000.0,1800000.00,0,0,0,0,0,0,3260000
2191,0.0,0,0.0,0,0,0,0,0,356000.00,0.0,...,0.0,538000.0,2066666.66,0,0,0,0,0,0,0
4261,0.0,0,0.0,0,0,0,0,0,0.00,0.0,...,342000.0,0.0,0.00,16332000,0,0,0,0,0,0
3493,0.0,0,0.0,0,0,0,0,0,1838333.34,0.0,...,0.0,294000.0,0.00,6502000,0,0,0,0,0,0
3340,0.0,0,0.0,0,0,1940000,0,0,4870000.00,0.0,...,200000.0,0.0,4000000.00,0,0,9200000,0,0,0,4420000
156,0.0,0,0.0,0,0,740444,0,0,600000.00,0.0,...,0.0,0.0,0.00,0,0,0,0,0,0,0
4040,0.0,0,0.0,0,0,0,0,0,0.00,0.0,...,0.0,0.0,0.00,0,0,0,0,0,0,0


In [16]:
print('Column searching (second time) ...')
df_new = bf_search(df_new, df_cand_col)

Column searching (second time) ...


In [17]:
print('new shape', df_new.shape)
train.loc[df_new.index, ["ID","target"]+df_new.columns.tolist()]

new shape (22, 40)


,ID,target,ced6a7e91,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,...,b85fa8b27,2155f5e16,794e93ca6,070f95c99,939f628a7,7e814a30d,a6e871369,0dc4d6c7d,bc70cbc26,aca228668
2848,a70090dc7,400000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,532000.00,...,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00,791200.00,334000.00
1582,5b74875d4,707000.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,565142.86,...,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00,791200.00
3509,ca6c4ea0d,547000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,0.00,...,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34,1338000.00
626,242407fc6,11115200.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,1432000.00,...,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00,1025333.34
2191,7ff66b22b,2253333.34,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,320000.00,...,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00,600000.00
4261,f43a48254,25300000.00,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,1359000.00,...,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66,4200000.00
3493,c9573f4ac,819750.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,380000.00,...,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00,242666.66
3340,c14136677,40000.00,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,900000.00,...,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00,113600.00
156,08f544828,25100000.00,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,500000.00,...,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00,281000.00
4040,e77e32574,22134000.00,371200.00,236857.14,2400000.00,1234666.66,1614909.10,819500.00,831000.00,0.00,...,332000.00,57714.28,346444.44,62000.00,0.00,0.00,514666.66,0.00,440500.00,132200.00


In [18]:
print(f'Row indexes({df_new.shape[0]})\n', df_new.index.values.tolist())
print(f'Column indexes({df_new.shape[1]})\n', df_new.columns.values.tolist())

Row indexes(22)
 [2848, 1582, 3509, 626, 2191, 4261, 3493, 3340, 156, 4040, 1681, 1311, 1125, 774, 2700, 3742, 3843, 2162, 1778, 2217, 4109, 1918]
Column indexes(40)
 ['ced6a7e91', '9df4daa99', '83c3779bf', 'edc84139a', 'f1e0ada11', '73687e512', 'aa164b93b', '342e7eb03', 'cd24eae8a', '8f3740670', '2b2a10857', 'a00adf70e', '3a48a2cd2', 'a396ceeb9', '9280f3d04', 'fec5eaf1a', '5b943716b', '22ed6dba3', '5547d6e11', 'e222309b0', '5d3b81ef8', '1184df5c2', '2288333b4', 'f39074b55', 'a8b721722', '13ee58af1', 'fb387ea33', '4da206d28', 'ea4046b8d', 'ef30f6be5', 'b85fa8b27', '2155f5e16', '794e93ca6', '070f95c99', '939f628a7', '7e814a30d', 'a6e871369', '0dc4d6c7d', 'bc70cbc26', 'aca228668']


In [19]:
for i, c in enumerate(df_new.columns):
    print(
        'No.', i, 'Column Name', c, 
        'subset count',
        (df_new[c].values==1563411.76).sum(), 
        'train count',
        (train[c].values==1563411.76).sum()
    )

No. 0 Column Name ced6a7e91 subset count 0 train count 0
No. 1 Column Name 9df4daa99 subset count 0 train count 0
No. 2 Column Name 83c3779bf subset count 0 train count 0
No. 3 Column Name edc84139a subset count 0 train count 0
No. 4 Column Name f1e0ada11 subset count 0 train count 0
No. 5 Column Name 73687e512 subset count 0 train count 0
No. 6 Column Name aa164b93b subset count 0 train count 0
No. 7 Column Name 342e7eb03 subset count 0 train count 0
No. 8 Column Name cd24eae8a subset count 0 train count 0
No. 9 Column Name 8f3740670 subset count 0 train count 0
No. 10 Column Name 2b2a10857 subset count 0 train count 0
No. 11 Column Name a00adf70e subset count 0 train count 0
No. 12 Column Name 3a48a2cd2 subset count 0 train count 0
No. 13 Column Name a396ceeb9 subset count 0 train count 0
No. 14 Column Name 9280f3d04 subset count 0 train count 0
No. 15 Column Name fec5eaf1a subset count 0 train count 0
No. 16 Column Name 5b943716b subset count 0 train count 0
No. 17 Column Name 22ed6

In [20]:
res_cnt = dict((c, (train[c].values==1563411.76).sum()) for c in train.columns[2:])
res_cnt = pd.DataFrame.from_dict(res_cnt, orient='index', columns=['strange_number_cnt'])
res_cnt = res_cnt.sort_values('strange_number_cnt', 0, False)
res_cnt.head(50).T

,0ff32eb98,c5a231d81,c47340d97,adb64ff71,190db8488,0572565c2,91f701ba2,fc99f9426,1db387535,6619d81fc,...,7f0d863ba,b22eb2036,e70581bed,adbe5fd20,c30399758,48df886f9,ea3f3029c,b88e5de84,1aa44bdf8,8e978ee65
strange_number_cnt,36,35,34,33,32,31,30,30,29,28,...,0,0,0,0,0,0,0,0,0,0


In [22]:
res_cnt.head(10)

,strange_number_cnt
0ff32eb98,36
c5a231d81,35
c47340d97,34
adb64ff71,33
190db8488,32
0572565c2,31
91f701ba2,30
fc99f9426,30
1db387535,29
6619d81fc,28


In [24]:
for i, c in enumerate(df_new.T.columns):
    print(
        'No.', i, 'Row Name', c, 
        'subset count',
        (df_new.T[c].values==1563411.76).sum(), 
        'train count',
        (train.T[c].values==1563411.76).sum()
    )

No. 0 Row Name 2848 subset count 0 train count 0
No. 1 Row Name 1582 subset count 0 train count 0
No. 2 Row Name 3509 subset count 0 train count 0
No. 3 Row Name 626 subset count 0 train count 0
No. 4 Row Name 2191 subset count 0 train count 0
No. 5 Row Name 4261 subset count 0 train count 0
No. 6 Row Name 3493 subset count 0 train count 0
No. 7 Row Name 3340 subset count 0 train count 0
No. 8 Row Name 156 subset count 0 train count 0
No. 9 Row Name 4040 subset count 0 train count 0
No. 10 Row Name 1681 subset count 0 train count 0
No. 11 Row Name 1311 subset count 0 train count 0
No. 12 Row Name 1125 subset count 0 train count 0
No. 13 Row Name 774 subset count 0 train count 0
No. 14 Row Name 2700 subset count 0 train count 0
No. 15 Row Name 3742 subset count 0 train count 0
No. 16 Row Name 3843 subset count 0 train count 0
No. 17 Row Name 2162 subset count 0 train count 0
No. 18 Row Name 1778 subset count 0 train count 0
No. 19 Row Name 2217 subset count 0 train count 0
No. 20 Row Na

In [25]:
tmp = train.iloc[:, 2:].values
res_t_cnt = dict((idx, (tmp[i, :]==1563411.76).sum()) for i,idx in enumerate(train.index))
res_t_cnt = pd.DataFrame.from_dict(res_t_cnt, orient='index', columns=['strange_number_cnt'])
res_t_cnt = res_t_cnt.sort_values('strange_number_cnt', 0, False)
res_t_cnt.head(50).T

,1757,3809,511,3798,625,3303,4095,1283,4209,1696,...,2975,2989,2983,2976,2985,2973,2977,2978,2981,2987
strange_number_cnt,39,38,37,36,35,34,33,31,30,29,...,0,0,0,0,0,0,0,0,0,0


In [26]:
head_row_indexes = res_t_cnt[res_t_cnt['strange_number_cnt']>24].index.tolist()
head_row_indexes

[1757, 3809, 511, 3798, 625, 3303, 4095, 1283, 4209, 1696, 3511, 816, 245]

In [30]:
mask = res_t_cnt['strange_number_cnt']>0 
mask&=res_t_cnt['strange_number_cnt']<8
tail_row_indexes = res_t_cnt.loc[mask].index.tolist()
tail_row_indexes

[4384, 3418, 4396, 921, 3176, 650]

In [28]:
pd.concat([
    train.loc[head_row_indexes, ['target']+df_new.columns.tolist()], 
    train.loc[df_new.index, ['target']+df_new.columns.tolist()],
    train.loc[tail_row_indexes, ['target']+df_new.columns.tolist()], 
])

,target,ced6a7e91,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,cd24eae8a,...,b85fa8b27,2155f5e16,794e93ca6,070f95c99,939f628a7,7e814a30d,a6e871369,0dc4d6c7d,bc70cbc26,aca228668
1757,115636.36,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
3809,834800.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
511,296444.44,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
3798,247166.66,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
625,550000.00,179000.00,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
3303,3076666.66,120400.00,179000.00,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
4095,440000.00,792750.00,120400.00,179000.00,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
1283,1600000.00,350000.00,609000.00,792750.00,120400.00,179000.00,800000.00,168000.00,210000.00,273000.00,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
4209,466461.54,0.00,350000.00,609000.00,792750.00,120400.00,179000.00,800000.00,168000.00,210000.00,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66
1696,3147200.00,0.00,0.00,350000.00,609000.00,792750.00,120400.00,179000.00,800000.00,168000.00,...,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66


In [31]:
df_new = pd.concat([
    train.loc[head_row_indexes, df_new.columns.tolist()], 
    train.loc[df_new.index, df_new.columns.tolist()],
    train.loc[tail_row_indexes, df_new.columns.tolist()], 
])

In [32]:
def row_bf_search(df_new):
    df_new = df_new.T.copy()
    df_cand_row = train[df_new.index].T.copy()
    print('Row searching ...')
    df_new = bf_search(df_new, df_cand_row)
    df_new = df_new.T.copy()
    return df_new
def column_bf_search(df_new):
    df_cand_col = train.loc[df_new.index, :]
    df_cand_col = df_cand_col.iloc[:, 2:]
    print('Column searching ...')
    df_new = bf_search(df_new, df_cand_col)
    return df_new

In [33]:
df_new = column_bf_search(df_new)

Column searching ...


In [34]:
df_new = row_bf_search(df_new)

Row searching ...


In [35]:
res_cnt[:10]

,strange_number_cnt
0ff32eb98,36
c5a231d81,35
c47340d97,34
adb64ff71,33
190db8488,32
0572565c2,31
91f701ba2,30
fc99f9426,30
1db387535,29
6619d81fc,28


In [36]:
train.loc[df_new.index, ['target']+df_new.columns.tolist()+['fc99f9426','91f701ba2'] + res_cnt.index.values[:6].tolist()[::-1]]

,target,ced6a7e91,9df4daa99,83c3779bf,edc84139a,f1e0ada11,73687e512,aa164b93b,342e7eb03,cd24eae8a,...,bc70cbc26,aca228668,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
1757,115636.36,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3809,834800.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
511,296444.44,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3798,247166.66,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
625,550000.00,179000.00,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
3303,3076666.66,120400.00,179000.00,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4095,440000.00,792750.00,120400.00,179000.00,800000.00,168000.00,210000.00,273000.00,1964666.66,1964666.66,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1283,1600000.00,350000.00,609000.00,792750.00,120400.00,179000.00,800000.00,168000.00,210000.00,273000.00,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
4209,466461.54,0.00,350000.00,609000.00,792750.00,120400.00,179000.00,800000.00,168000.00,210000.00,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76
1696,3147200.00,0.00,0.00,350000.00,609000.00,792750.00,120400.00,179000.00,800000.00,168000.00,...,1964666.66,1964666.66,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76,1563411.76


In [37]:
train.loc[df_new.index[:8], ['f190486d6', '58e2e02e6']].T

,1757,3809,511,3798,625,3303,4095,1283
f190486d6,1015000.00,540000.0,115636.36,834800.00,296444.44,247166.66,550000.00,440000.00
58e2e02e6,1563411.76,1015000.0,540000.00,115636.36,834800.00,296444.44,247166.66,3076666.66


In [38]:
train.loc[df_new.index[-2:], df_new.columns.tolist()[-2:]+['fc99f9426', '91f701ba2']+res_cnt.index.values[:6].tolist()[::-1]]

,bc70cbc26,aca228668,fc99f9426,91f701ba2,0572565c2,190db8488,adb64ff71,c47340d97,c5a231d81,0ff32eb98
3176,1964666.66,1964666.66,247166.66,296444.44,834800.00,115636.36,540000.00,1015000.0,1563411.76,1563411.76
650,273000.00,1964666.66,550000.00,247166.66,296444.44,834800.00,115636.36,540000.0,1015000.00,1563411.76


In [39]:
df_new_new = train.loc[df_new.index, df_new.columns.tolist()+['fc99f9426','91f701ba2'] + res_cnt.index.values[:6].tolist()[::-1]]
df_new_new.shape

(41, 48)

In [ ]:
print(f'Row indexes({df_new_new.shape[0]})\n', df_new_new.index.values.tolist())
print(f'Column indexes({df_new_new.shape[1]})\n', df_new_new.columns.values.tolist())

In [ ]:
train.loc[df_new_new.index, ['ID', 'target']+df_new_new.columns.tolist()]